# 3D Basement LRD

In [1]:
DATADIR='data/' # Directory with the data
FIGURESDIR='figures/' # Figures produced

In [2]:
import functions
from functions import *

deltacontourn=pd.read_csv(DATADIR+'deltacontourn.csv')
xyzcontourn=coordinates(DATADIR+'deltacontourn.csv',[0,1,2])

contourn_bounds=bounds(xyzcontourn)

contourn_poly=Polygon(zip(xyzcontourn[0],xyzcontourn[1]))

We read the data from those boreholes that reach the basement.

In [3]:
basement=pd.read_csv(DATADIR+'basement.csv')

Now we apply the function 'coordinates' to extract the coordinates from the data.

In [4]:
xyzbasement=coordinates(DATADIR+'basement.csv',[1,2,3])

The function `nearby` reduces de coordinates to those inside the delta contour.

In [5]:
xyzbasement_near=nearby(xyzbasement,contourn_poly,500)

We are going to adapt the bounds keeping in mind the bound given by the data in the basement, so first we apply the `bounds` function to the basement and then we use a new function `bounds_join(b1,b2)` which calculates new bounds of two lists of bounds.

In [6]:
basement_bounds=bounds(xyzbasement)

In [7]:
new_bounds=bounds_join(contourn_bounds,basement_bounds)

We apply the interpolation.

In [8]:
basement_itp=interpolation(xyzbasement,100,new_bounds)

We want to represent the basement surface only within the delta contour. We have defined a 'slice' function to slice data, such as the output of the 'interpolation' function, and take only those points in a polynomial region. We apply this function to the `basement_itp` list.

In [9]:
cbasement_itp=cutting(basement_itp,contourn_poly,500)

We also want to add a mark at the position of each borehole at height 7.

In [10]:
xyzdat=coordinates(DATADIR+'boreholes.csv',[1,2,3])
xyzdat_near=nearby(xyzdat,contourn_poly,300)

The 'data_p' function is now used to obtain the data for the points in the figure.

In [11]:
data_points=data_p([xyzdat_near,xyzbasement_near],['boreholes location','basement points'],['darkblue','red'],['cross','circle'],2)

To draw the basement surface we use the plotly.graph_objects fuction 'Surface'.

In [12]:
fig=go.Figure(data=data_points)
fig.add_trace(go.Scatter3d(x=xyzcontourn[0],y=xyzcontourn[1],z=xyzcontourn[2], mode="lines",
                           line_width=5,
                           name='Delta Contour',
                           marker = dict(
                               size = 4,
                               color = 'black'
                               )
                          )
             )

fig.add_trace(go.Surface(z=cbasement_itp[0],x=cbasement_itp[1],y=cbasement_itp[2], 
                        opacity = 0.7,
                        colorscale='brwnyl',
                        name='superficie basemento',
                        #label='superficie basemento',
                        showscale=False
                        )             
             )

fig.update_layout( title="Pliocene basement Llobregat Delta, Z scale is x 50",
    scene=dict(aspectratio=dict(x=2, y=2, z=0.5),
               xaxis = dict(range=[contourn_bounds[0]-2000,contourn_bounds[1]+2000],),
               yaxis = dict(range=[contourn_bounds[2]-2000,contourn_bounds[3]+2000])
                            )
                 )

#fig.show()
go_offline.plot(fig,filename=FIGURESDIR+'3D_Basement_LRD.html',validate=True, auto_open=False)

'figures/3D_Basement_LRD.html'